# Семинар 6: Первое знакомство со Scikit-learn. Простая линейная регрессия.

**Цели семинара:**
1.  Познакомиться с основной философией библиотеки `scikit-learn`.
2.  Изучить базовый API на примере `fit`, `predict`.
3.  Научиться готовить данные к обучению: загрузка, анализ, разделение на выборки (`train_test_split`).
4.  Обучить свою первую модель `LinearRegression`.
5.  Оценить качество модели с помощью метрик (RMSE, MAE, R²) и визуального анализа остатков.

## 1. Введение в Scikit-Learn

**Scikit-learn** — это де-факто стандарт для классического машинного обучения в Python. Её популярность обусловлена несколькими причинами:

*   **Единообразный API:** Все модели (оценщики, *estimators*) имеют схожие методы: `.fit()` для обучения, `.predict()` для предсказания, `.score()` для оценки и т.д. Это позволяет легко заменять один алгоритм на другой.
*   **Широкий набор инструментов:** Включает в себя всё необходимое: алгоритмы, функции для подготовки данных, метрики качества.
*   **Отличная документация:** С множеством примеров и подробными объяснениями.

### Основной рабочий процесс (Estimator API)

1.  **Импорт:** `from sklearn.module import Model`
2.  **Инстанцирование:** `model = Model(hyperparameter=value)`
3.  **Обучение:** `model.fit(X_train, y_train)`
4.  **Предсказание:** `predictions = model.predict(X_test)`
5.  **Оценка:** `score = metric_function(y_test, predictions)`

Сегодня мы пройдем все эти шаги на практике.

## 2. Задача: Предсказание продаж на основе рекламы

Мы будем работать с популярным набором данных `Advertising.csv`, который содержит информацию о продажах продукта в зависимости от рекламного бюджета, потраченного на ТВ, радио и газеты.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Загружаем данные
df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/Advertising.csv')

# Смотрим на первые 5 строк
df.head()

### 2.1. Исследовательский анализ данных (EDA)

Прежде чем обучать модель, всегда нужно посмотреть на данные глазами. Построим pairplot, чтобы визуально оценить зависимости между переменными.

In [ ]:
sns.pairplot(df)
plt.show()

**Вывод из pairplot:** Видна сильная положительная линейная зависимость между бюджетом на ТВ-рекламу (`TV`) и продажами (`sales`). Зависимость от `radio` тоже есть, но с большим разбросом. `newspaper` выглядит наименее связанным с продажами.

Давайте выделим признаки (X) и целевую переменную (y).

In [ ]:
# Признаки
X = df.drop('Sales', axis=1)

# Целевая переменная
y = df['Sales']

### 2.2. Разделение данных на обучающую и тестовую выборки

Это самый важный шаг для объективной оценки модели. Мы отложим часть данных (тестовую выборку), чтобы проверить на ней нашу модель после обучения. Для этого используем функцию `train_test_split`.

In [ ]:
from sklearn.model_selection import train_test_split

# test_size=0.3 означает, что 30% данных уйдет в тестовую выборку
# random_state=42 обеспечивает воспроизводимость результата (одинаковое "случайное" разбиение)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 3. Обучение модели линейной регрессии

In [ ]:
from sklearn.linear_model import LinearRegression

# Шаг 1: Инстанцирование модели
model = LinearRegression()

# Шаг 2: Обучение модели на обучающих данных
model.fit(X_train, y_train)

### 3.1. Интерпретация коэффициентов

Модель обучена! Она нашла оптимальные коэффициенты для нашего уравнения регрессии. Давайте посмотрим на них.

In [ ]:
print("Свободный член (intercept, w0):", model.intercept_)

# Создадим DataFrame для удобного просмотра весов
coeffs = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print("\nВеса для каждого признака (w1, w2, ...):")
print(coeffs)

**Как это интерпретировать?**
*   **Intercept (2.85):** Если мы не потратим ни рубля на рекламу, наша модель предсказывает продажи на уровне 2.69 единиц.
*   **TV (0.04):** Каждый дополнительный рубль, вложенный в ТВ-рекламу, при прочих равных увеличивает продажи на 0.04 единиц.
*   **radio (0.199):** Каждый дополнительный рубль на радио-рекламу дает прирост в 0.199 единиц.
*   **newspaper (0.0001):** Влияние газет, по мнению модели, минимально.

## 4. Оценка качества модели

Теперь, когда модель обучена, оценим ее качество на **тестовой выборке**, которую модель еще не видела.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Шаг 3: Делаем предсказания на тестовых данных
y_pred = model.predict(X_test)

# Шаг 4: Оцениваем качество с помощью метрик
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE (Mean Absolute Error): {mae:.2f}")
print(f"MSE (Mean Squared Error): {mse:.2f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"R^2 (Коэффициент детерминации): {r2:.2f}")

**Интерпретация метрик:**
*   **RMSE (1.93):** В среднем, наша модель ошибается в предсказании продаж на 1.95 единицы. Учитывая, что средние продажи составляют около 14, это неплохой результат.
*   **R² (0.86):** Наша модель объясняет 86% изменчивости (дисперсии) в данных о продажах. Это очень хороший показатель.

### 4.1. Анализ остатков

Проверим, нет ли в ошибках модели каких-либо скрытых закономерностей. В идеале остатки должны быть распределены случайно вокруг нуля.

In [ ]:
residuals = y_test - y_pred

sns.scatterplot(x=y_test, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('График остатков')
plt.xlabel('Истинные значения y_test')
plt.ylabel('Остатки')
plt.show()

**Вывод:** График выглядит хорошо. Точки распределены хаотично, без явных паттернов (например, параболы), что говорит о том, что линейная модель в данном случае является адекватной.

### 4.2. Углубленный анализ остатков: ищем зависимости

Еще один полезный диагностический инструмент — построить графики остатков относительно каждого из исходных признаков. Если модель полностью "впитала" всю информацию из признака, то в ошибках не должно остаться никакой структуры, связанной с этим признаком.

In [ ]:
# Создаем фигуру с тремя графиками
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Графики остатков по каждому признаку', fontsize=16)

# Остатки vs TV
sns.scatterplot(ax=axes[0], x=X_test['TV'], y=residuals)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_xlabel('Бюджет на TV')
axes[0].set_ylabel('Остатки')

# Остатки vs Radio
sns.scatterplot(ax=axes[1], x=X_test['Radio'], y=residuals)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Бюджет на Radio')
axes[1].set_ylabel('')

# Остатки vs Newspaper
sns.scatterplot(ax=axes[2], x=X_test['Newspaper'], y=residuals)
axes[2].axhline(y=0, color='r', linestyle='--')
axes[2].set_xlabel('Бюджет на Newspaper')
axes[2].set_ylabel('')

plt.show()

**Вывод:** Во всех трех случаях остатки выглядят как случайный "шум", равномерно распределенный вокруг нуля. Это еще раз подтверждает, что простая линейная модель хорошо подходит для этих данных и не упускает каких-либо очевидных нелинейных зависимостей. Если бы, например, на графике "Остатки vs TV" мы увидели параболу, это был бы сильный сигнал к тому, что в модель нужно добавить признак `TV^2`.

## Заключение

На этом семинаре мы прошли полный цикл построения простой модели машинного обучения с использованием `scikit-learn`: от загрузки данных до обучения, оценки и диагностики. Мы убедились, что линейная регрессия хорошо справляется с данной задачей.